In [450]:
import math
import pandas as pd
import torch
from torch import nn
from d2l import torch as d2l

In [451]:
#@save
class PositionWiseFFN(nn.Module):
    """基于位置的前馈网络"""
    def __init__(self, ffn_num_input, ffn_num_hiddens, ffn_num_outputs,
                 **kwargs):
        super(PositionWiseFFN, self).__init__(**kwargs)
        self.dense1 = nn.Linear(ffn_num_input, ffn_num_hiddens)
        self.relu = nn.ReLU()
        self.dense2 = nn.Linear(ffn_num_hiddens, ffn_num_outputs)

    def forward(self, X):
        return self.dense2(self.relu(self.dense1(X)))

In [452]:
ffn = PositionWiseFFN(4, 4, 8)
ffn.eval()
ffn(torch.ones((2, 3, 4)))[0]

tensor([[ 0.3349, -0.2140, -0.3565,  0.3605,  0.3905,  0.4985,  0.2844,  0.3959],
        [ 0.3349, -0.2140, -0.3565,  0.3605,  0.3905,  0.4985,  0.2844,  0.3959],
        [ 0.3349, -0.2140, -0.3565,  0.3605,  0.3905,  0.4985,  0.2844,  0.3959]],
       grad_fn=<SelectBackward0>)

In [453]:
ln = nn.LayerNorm(2)
bn = nn.BatchNorm1d(2)
X = torch.tensor([[1, 2], [2, 3]], dtype=torch.float32)
# 在训练模式下计算X的均值和方差
print('layer norm:', ln(X), '\nbatch norm:', bn(X))

layer norm: tensor([[-1.0000,  1.0000],
        [-1.0000,  1.0000]], grad_fn=<NativeLayerNormBackward0>) 
batch norm: tensor([[-1.0000, -1.0000],
        [ 1.0000,  1.0000]], grad_fn=<NativeBatchNormBackward0>)


In [454]:
#@save
class AddNorm(nn.Module):
    """残差连接后进行层规范化"""
    def __init__(self, normalized_shape, dropout, **kwargs):
        super(AddNorm, self).__init__(**kwargs)
        self.dropout = nn.Dropout(dropout)
        self.ln = nn.LayerNorm(normalized_shape)

    def forward(self, X, Y):
        return self.ln(self.dropout(Y) + X)

In [455]:
add_norm = AddNorm([3, 4], 0.5)
add_norm.eval()
add_norm(torch.ones((2, 3, 4)), torch.ones((2, 3, 4))).shape

torch.Size([2, 3, 4])

In [456]:
#@save
class EncoderBlock(nn.Module):
    """Transformer编码器块"""
    def __init__(self, key_size, query_size, value_size, num_hiddens,
                 norm_shape, ffn_num_input, ffn_num_hiddens, num_heads,
                 dropout, use_bias=False, **kwargs):
        super(EncoderBlock, self).__init__(**kwargs)
        self.attention = d2l.MultiHeadAttention(
            key_size, query_size, value_size, num_hiddens, num_heads, dropout,
            use_bias)
        self.addnorm1 = AddNorm(norm_shape, dropout)
        self.ffn = PositionWiseFFN(
            ffn_num_input, ffn_num_hiddens, num_hiddens)
        self.addnorm2 = AddNorm(norm_shape, dropout)

    def forward(self, X, valid_lens):
        Y = self.addnorm1(X, self.attention(X, X, X, valid_lens))
        return self.addnorm2(Y, self.ffn(Y))

In [457]:
X = torch.ones((2, 100, 24))
valid_lens = torch.tensor([3, 2])
encoder_blk = EncoderBlock(24, 24, 24, 24, [100, 24], 24, 48, 8, 0.5)
encoder_blk.eval()
encoder_blk(X, valid_lens).shape

torch.Size([2, 100, 24])

In [458]:
#@save
class TransformerEncoder(d2l.Encoder):
    """Transformer编码器"""
    def __init__(self, vocab_size, key_size, query_size, value_size,
                 num_hiddens, norm_shape, ffn_num_input, ffn_num_hiddens,
                 num_heads, num_layers, dropout, use_bias=False, **kwargs):
        super(TransformerEncoder, self).__init__(**kwargs)
        self.num_hiddens = num_hiddens
        # vocab_size 是在 embedding 层中进行 one-hot 编码步骤用到的
        # num_hiddens 是把输入到 embedding 的最后一个维度的每个元素展开成 num_hiddens 长的向量
        # num_hiddens 代表每个词元 使用 num_hiddens 长度的向量表示
        self.embedding = nn.Embedding(vocab_size, num_hiddens)
        # 对输入的数据添加位置编码
        self.pos_encoding = d2l.PositionalEncoding(num_hiddens, dropout)
        self.blks = nn.Sequential()
        for i in range(num_layers):
            self.blks.add_module("block"+str(i),
                EncoderBlock(key_size, query_size, value_size, num_hiddens,
                             norm_shape, ffn_num_input, ffn_num_hiddens,
                             num_heads, dropout, use_bias))

    def forward(self, X, valid_lens, *args):
        # 因为位置编码值在-1和1之间，
        # 因此嵌入值乘以嵌入维度的平方根进行缩放，
        # 然后再与位置编码相加。
        X = self.pos_encoding(self.embedding(X) * math.sqrt(self.num_hiddens))
        self.attention_weights = [None] * len(self.blks)
        for i, blk in enumerate(self.blks):
            X = blk(X, valid_lens)
            self.attention_weights[i] = blk.attention.attention.attention_weights

        # X 是经过 Encoder 汇聚之后的结果
        # X 包含了所有的输入信息，但又不是全都包含在了一个像是 RNN 的 state 矩阵中
        # 把 X 丢给 Encoder 的第二个注意力层后，由这个注意力层来选择使用哪些信息
        return X

In [459]:
encoder = TransformerEncoder(
    200, 24, 24, 24, 24, [100, 24], 24, 48, 8, 2, 0.5)
encoder.eval()
encoder(torch.ones((2, 100), dtype=torch.long), valid_lens).shape

torch.Size([2, 100, 24])

In [460]:
class DecoderBlock(nn.Module):
    """解码器中第i个块"""
    def __init__(self, key_size, query_size, value_size, num_hiddens,
                 norm_shape, ffn_num_input, ffn_num_hiddens, num_heads,
                 dropout, i, **kwargs):
        super(DecoderBlock, self).__init__(**kwargs)
        self.i = i  # 存一下自己是第几个块
        self.attention1 = d2l.MultiHeadAttention(
            key_size, query_size, value_size, num_hiddens, num_heads, dropout)
        self.addnorm1 = AddNorm(norm_shape, dropout)  # norm 应该等于 num_hiddens 吧
        self.attention2 = d2l.MultiHeadAttention(
            key_size, query_size, value_size, num_hiddens, num_heads, dropout)
        self.addnorm2 = AddNorm(norm_shape, dropout)
        self.ffn = PositionWiseFFN(ffn_num_input, ffn_num_hiddens,  # ffn_num_input 应该等于 num_hiddens 把
                                   num_hiddens)
        self.addnorm3 = AddNorm(norm_shape, dropout)  # norm_shape 应该等于 num_hiddens 吧

    def forward(self, X, state):
        # X (batch_size, num_steps, embedding_size=num_hiddens)
        # decoder_block 拿到的输入 X 是原始样本经过 embedding 后的产物
        enc_outputs, enc_valid_lens = state[0], state[1]
        # 训练阶段，输出序列的所有词元都在同一时间处理，
        # 因此state[2][self.i]初始化为None。
        # 预测阶段，输出序列是通过词元一个接着一个解码的，
        # 因此state[2][self.i]包含着直到当前时间步第i个块解码的输出表示

        # state[2] 中的 [self.i] 并不是代表第 i 个时刻的东西
        # 而是 第i个 Decoder 块的输入数据
        # 在第 0 号时间步的数据进行前向运算的时候 state[2][...] 所有位置都是 None
        # 因此在计算第 0 号时间步前向运算时，是第一次在 state[2][...] 中填充内容(对于每个样本序列而言，仅填充了一行)
        # 接着计算后边的时间步的数值进行前向运算的时候 只需要把当前层的输入往下进行拼接即可

        # state[2] 对于训练的过程没有什么帮助
        # 在预测的时候才有帮助，因为预测的时候，每个时间步产生的每一个 Decoder-Block 的输出是一个时间步一个时间步产生的
        # 而每次让 q 和所有的 k 做运算的时候，是需要把之前上一层的 Decoder-Block 产生的内容作为下一层 Decoder-Block 中
        # 的 Attention 机制的 key-values pairs 的输入的，所以需要把之前时间步产生的 key-value 对给保存下来
        
        # 这同时也解释了，为什么在预测的时候不需要 valid_lens, 因为在 state[2] 中保存的 key_value 也就只有那么点
        # 也就只有目前能够看到的序列产生的输出作为 key_value pairs 而后边的 key_value pairs 根本没存进去呢
        
        # 另外也能够解释为什么在训练的时候需要使用 valid_lens，因为在训练的时候一次性的，所有时间步的值都可以并行的作为输入
        # 而 Self-Attention 计算第 i 个 q 的输出并不需要非得把前边的那些 q 都计算完了才能计算第 i 个 q
        # 仅需要有所有的 key_values pairs 就可以计算任意一个 q 产生的输出了，这个不像 RNN 只能串行计算，而是可以并行计算
        # 因此，一次性就输入进模型所有的 key_values pairs 了，这时候计算第 i 个 q 的时候与每一个 k 都进行了 dot product
        # 但是，按照预测的逻辑来看，计算第 i 个 q 的 dot product 的时候，i 到 num_steps 的这些 q 产生的输出还不存在呢
        # 但是在训练的时候这些 q 却可以被看到，因此就需要 valid_lens 来指明当前的 q 只能看到前多少个 key_values pairs

        if state[2][self.i] is None:
            key_values = X
        else:
            key_values = torch.cat((state[2][self.i], X), axis=1)
        state[2][self.i] = key_values
        if self.training:  # 应该是 net.train() 起的作用；把 self.training 设置为 True
            batch_size, num_steps, _ = X.shape  # _ 是 embedding_size
            # dec_valid_lens的开头:(batch_size,num_steps),
            # 其中每一行是[1,2,...,num_steps]
            # dec_valid_lens 是一个矩阵
            # 做注意力机制的时候，每一个序列中的第 i 个 q 只能和这个序列的前 i 个 k 进行得分计算
            # 因此就是 [1,2,...,num_steps]
            dec_valid_lens = torch.arange(
                1, num_steps + 1, device=X.device).repeat(batch_size, 1)
        else:
            dec_valid_lens = None

        # 自注意力
        # 做注意力机制的时候，每一个序列中的第 i 个 q 只能和这个序列的前 i 个 k 进行得分计算
        X2 = self.attention1(X, key_values, key_values, dec_valid_lens)
        Y = self.addnorm1(X, X2)
        # 编码器－解码器注意力。
        # enc_outputs的开头:(batch_size,num_steps,num_hiddens)
        Y2 = self.attention2(Y, enc_outputs, enc_outputs, enc_valid_lens)
        Z = self.addnorm2(Y, Y2)
        return self.addnorm3(Z, self.ffn(Z)), state

In [461]:
decoder_blk = DecoderBlock(24, 24, 24, 24, [100, 24], 24, 48, 8, 0.5, 0)
decoder_blk.eval()
X = torch.ones((2, 100, 24))
state = [encoder_blk(X, valid_lens), valid_lens, [None]]
decoder_blk(X, state)[0].shape

torch.Size([2, 100, 24])

In [462]:
class TransformerDecoder(d2l.AttentionDecoder):
    def __init__(self, vocab_size, key_size, query_size, value_size,
                 num_hiddens, norm_shape, ffn_num_input, ffn_num_hiddens,
                 num_heads, num_layers, dropout, **kwargs):
        super(TransformerDecoder, self).__init__(**kwargs)
        self.num_hiddens = num_hiddens
        self.num_layers = num_layers
        self.embedding = nn.Embedding(vocab_size, num_hiddens)
        self.pos_encoding = d2l.PositionalEncoding(num_hiddens, dropout)
        self.blks = nn.Sequential()
        for i in range(num_layers):
            self.blks.add_module("block"+str(i),
                DecoderBlock(key_size, query_size, value_size, num_hiddens,
                             norm_shape, ffn_num_input, ffn_num_hiddens,
                             num_heads, dropout, i))
        self.dense = nn.Linear(num_hiddens, vocab_size)

    def init_state(self, enc_outputs, enc_valid_lens, *args):
        return [enc_outputs, enc_valid_lens, [None] * self.num_layers]

    def forward(self, X, state):
        X = self.pos_encoding(self.embedding(X) * math.sqrt(self.num_hiddens))
        self._attention_weights = [[None] * len(self.blks) for _ in range (2)]
        for i, blk in enumerate(self.blks):  # i 代表的是第 i 号块，而 state[2][i] 确实是第 i 个块的输入，且是第 i-1 号块的输出
            X, state = blk(X, state)
            # 解码器自注意力权重
            self._attention_weights[0][i] = blk.attention1.attention.attention_weights
            # “编码器－解码器”自注意力权重
            self._attention_weights[1][i] = blk.attention2.attention.attention_weights
        return self.dense(X), state

    @property
    def attention_weights(self):
        return self._attention_weights

#### 训练

In [463]:
num_hiddens, num_layers, dropout, batch_size, num_steps = 32, 2, 0.1, 64, 10
lr, num_epochs, device = 0.005, 200, d2l.try_gpu()
ffn_num_input, ffn_num_hiddens, num_heads = 32, 64, 4
key_size, query_size, value_size = 32, 32, 32
norm_shape = [32]

train_iter, src_vocab, tgt_vocab = d2l.load_data_nmt(batch_size, num_steps)

# encoder = TransformerEncoder(
#     len(src_vocab), key_size, query_size, value_size, num_hiddens,
#     norm_shape, ffn_num_input, ffn_num_hiddens, num_heads,
#     num_layers, dropout)
# decoder = TransformerDecoder(
#     len(tgt_vocab), key_size, query_size, value_size, num_hiddens,
#     norm_shape, ffn_num_input, ffn_num_hiddens, num_heads,
#     num_layers, dropout)
net = d2l.EncoderDecoder(encoder, decoder)

# 编码器的输入是一个二维的样本矩阵 X 每一行都是一个句子
# 解码器的输入 dec_input 也是一个样本矩阵，每一行都是一个句子并且有开始标志 <bos> 是在训练的时候添加上的
d2l.train_seq2seq(net, train_iter, lr, num_epochs, tgt_vocab, device)

In [485]:
for X, X_valid_lens, Y, Y_valid_lens in train_iter:
    print(X.shape, ' and ', Y.shape)
    print(X[:3, :])
    print(Y[:3, :])
    print(src_vocab.idx_to_token[1])
    print(tgt_vocab.idx_to_token[1])
    print(X_valid_lens[:3])
    print(Y_valid_lens[:3])
    break

torch.Size([64, 10])  and  torch.Size([64, 10])
tensor([[ 7, 10,  4,  3,  1,  1,  1,  1,  1,  1],
        [30, 38,  4,  3,  1,  1,  1,  1,  1,  1],
        [ 9, 28,  5,  3,  1,  1,  1,  1,  1,  1]])
tensor([[  6,   7,   8,   4,   3,   1,   1,   1,   1,   1],
        [148,   5,   3,   1,   1,   1,   1,   1,   1,   1],
        [ 59,   5,   3,   1,   1,   1,   1,   1,   1,   1]])
<pad>
<pad>
tensor([4, 4, 4])
tensor([5, 3, 3])


#### 计算 BLEU

In [465]:
# engs = ['go .', "i lost .", 'he\'s calm .', 'i\'m home .']
# fras = ['va !', 'j\'ai perdu .', 'il est calme .', 'je suis chez moi .']
# for eng, fra in zip(engs, fras):
#     translation, dec_attention_weight_seq = d2l.predict_seq2seq(
#         net, eng, src_vocab, tgt_vocab, num_steps, device, True)
#     print(f'{eng} => {translation}, ',
#           f'bleu {d2l.bleu(translation, fra, k=2):.3f}')

In [466]:
# enc_attention_weights = torch.cat(net.encoder.attention_weights, 0).reshape((num_layers, num_heads,
#     -1, num_steps))
# enc_attention_weights.shape

In [467]:
# d2l.show_heatmaps(
#     enc_attention_weights.cpu(), xlabel='Key positions',
#     ylabel='Query positions', titles=['Head %d' % i for i in range(1, 5)],
#     figsize=(7, 3.5))

#### 理解 Layer-Norm 和 Batch-Norm

In [468]:
X = torch.tensor([
    [1, 2],
    [2, 3]
], dtype=torch.float32)

In [469]:
# 传入一个整数 2 
# 则参数被看作是一个只有一个元素 2 的 list
# 代表对最后一个维度上长度为 2 的每个向量分别做归一化
ln = nn.LayerNorm(2)
bn = nn.BatchNorm1d(2)

In [470]:
# Layer-Norm 是把矩阵中的每一行归一化（每行是一个样本中一个数值的向量表示）
# Batch-Norm 是把矩阵中的每一列归一化（每列是一个样本中多个数值的相同特征）
# 因此 波波老师 的视频中讲的 均值方差归一化 属于是 Batch-Norm
ln(X), bn(X)

(tensor([[-1.0000,  1.0000],
         [-1.0000,  1.0000]], grad_fn=<NativeLayerNormBackward0>),
 tensor([[-1.0000, -1.0000],
         [ 1.0000,  1.0000]], grad_fn=<NativeBatchNormBackward0>))

In [471]:
mean, var = torch.mean(X[0]), torch.var(X[0])

In [472]:
(1-mean) / var, (2-mean) / var

(tensor(-1.), tensor(1.))

In [473]:
mean, var = torch.mean(X[1]), torch.var(X[1])

In [474]:
(2-mean) / var, (3-mean) / var

(tensor(-1.), tensor(1.))

#### 理解 nn.Embedding

In [475]:
inputs = torch.tensor([
    [1, 2, 4],
    [2, 3, 5]
])

inputs

tensor([[1, 2, 4],
        [2, 3, 5]])

In [476]:
vocab_size = 6
num_hiddens = 7

"""
embedding : (不确定内部是不是这样实现的)
先对输入的序列中的每一个值做 one-hot
在把每一个 one-hot 转化成一个连续类型的向量（做了一个全连接层）

让 embedding 层学习到词和词之间的关系
经过 embedding 层输出的是一个实数向量
这样的一个向量表示的肯定不只是曾经 one-hot 表示的单个词的信息了
从而就学习到了词和词之间的关系（联系）信息
而且 embedding 的输出也不必和 词元个数(vocab_size) 相同，可以比 vocab_size 大，
因为它承载了更多的内容
"""

# 通过 Embedding 的方式把句子中的每一个词元向量化表示
embedding = nn.Embedding(vocab_size, num_hiddens)
embedding(inputs), embedding(inputs).shape

(tensor([[[ 0.5088, -1.5678, -1.1260,  1.3037,  0.2724, -0.6884,  1.7257],
          [-2.2114, -1.2187, -1.0790,  0.6996,  1.1503, -1.0259,  1.3760],
          [-0.8085,  0.4052, -0.3968, -1.7356, -0.2481,  1.3400, -1.2124]],
 
         [[-2.2114, -1.2187, -1.0790,  0.6996,  1.1503, -1.0259,  1.3760],
          [-0.7309,  0.8472, -0.3097,  0.3759, -0.0202, -0.6070, -0.3054],
          [-0.6857, -0.8837, -0.9190,  0.0105,  0.4702, -0.3287, -0.2345]]],
        grad_fn=<EmbeddingBackward0>),
 torch.Size([2, 3, 7]))

#### 理解 torch.cat

In [477]:
X1 = torch.tensor([
[
    [1, 2, 3, 4],
    [5, 6, 7, 8]
],
[
    [1, 2, 3, 4],
    [1, 2, 3, 4]
]
])

X2 = torch.tensor([
[
    [2, 2, 3, 4],
    [2, 6, 7, 8]
],
[
    [22, 2, 3, 4],
    [22, 2, 3, 4]
]
])

# 在第 1 号轴的方向上进行拼接，横着拼
# 对应模型中的 key_values 也是三维的
# (batch_size, num_key_values_pairs, num_hiddens)
# 每次都扩展 key_values，把之前输入的内容 concat 到 axis=1 上去
res = torch.cat((X1, X2), axis=1)
res

tensor([[[ 1,  2,  3,  4],
         [ 5,  6,  7,  8],
         [ 2,  2,  3,  4],
         [ 2,  6,  7,  8]],

        [[ 1,  2,  3,  4],
         [ 1,  2,  3,  4],
         [22,  2,  3,  4],
         [22,  2,  3,  4]]])

In [478]:
X1.shape, X2.shape

(torch.Size([2, 2, 4]), torch.Size([2, 2, 4]))

#### 理解 torch.arange

In [490]:
# 生成 1 到 3 之间的整数作为一个向量
# 把这个向量在 0 号维度复制 4 次 得到一个矩阵
# 把这个矩阵在 1 号维度复制 1 次 拼接起来
dec_valid_lens = torch.arange(1, 3).repeat(4, 1)
dec_valid_lens

tensor([[1, 2],
        [1, 2],
        [1, 2],
        [1, 2]])